In [1]:
project_directory=r"D:\PanCanAID-Batch1-202408"
patient_directory=r"D:\PanCanAID-Batch1-202408\2072"

# Libraries

In [2]:

#%pip install pydicom pandas matplotlib numpy
#%pip install GDCM pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg  # required packages for showing the dicom data using matplotlib
#%pip install ipywidgets # for scrollbar of multuple stacked slices


#libraires
import os
import xml.etree.ElementTree as ET
from typing import Optional, Dict, Any
import logging
import pydicom
# Configure logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
import pandas as pd 
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, IntSlider
from tqdm.notebook import tqdm


import pickle
import numpy as np
import vtk
from vtk.util import numpy_support


# functions
def read_dicom(dicom_path):
    """
    Reads a DICOM file from the given path.

    Parameters:
    - selected_SEGdicom_fullpath: The full path to the DICOM file.

    Returns:
    - The DICOM dataset if read successfully, otherwise None.
    """
    try:
        dicom_data = pydicom.dcmread(dicom_path)
        logging.info(f"Dicom data loaded from: {dicom_path}.")
        return dicom_data
    except Exception as e:
        logging.error(f"Error in reading '{dicom_path}': {e}")
        return None

# functions to use in future
def get_nested_element(dataset, tags):
    """
    Navigate through the DICOM dataset using a list of tags and return the final element.

    Parameters:
        dataset (pydicom.dataset.Dataset): The DICOM dataset.
        tags (list): A list of tuples representing the tags to navigate through.

    Returns:
        The final element in the DICOM dataset specified by the tags.
    """
    current_element = dataset
    for tag in tags:
        tag = pydicom.tag.Tag(tag)
        if tag in current_element:
            current_element = current_element[tag]
        else:
            raise KeyError(f"Tag {tag} not found in the dataset.")
        
        # If the current element is a sequence, assume we want the first item
        if isinstance(current_element, pydicom.sequence.Sequence):
            if len(current_element) > 0:
                current_element = current_element[0]
            else:
                raise ValueError(f"Sequence at tag {tag} is empty.")
    
    return current_element

# Functions

## Find all patients directory


In [ ]:
def list_directory_folders(directory):
    try:
        # Get the absolute path of the directory
        abs_directory = os.path.abspath(directory)
        
        # Check if the directory exists
        if not os.path.isdir(abs_directory):
            raise ValueError(f"The specified path '{abs_directory}' is not a valid directory.")
        
        # List all items in the directory
        all_items = os.listdir(abs_directory)
        
        # Filter out non-directories and create full paths
        folder_paths = [os.path.join(abs_directory, item) for item in all_items 
                        if os.path.isdir(os.path.join(abs_directory, item))]
        
        print("Number of folders:" , len(folder_paths))
        return folder_paths
    
    except PermissionError:
        print(f"Permission denied: Unable to access '{abs_directory}'.")
        return []
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return []

# Example usage (you can run this in a separate cell)
patient_directories = list_directory_folders(project_directory)


## Find all segmentations in a folder

In [ ]:
def assessors_path(patient_directory: str) -> Optional[str]:
    """
    This function takes the path to a patient directory and returns the path of the ASSESSORS directory if it exists.
    
    Parameters:
    - patient_directory (str): The directory path of the patient.
    
    Returns:
    - Optional[str]: The path to the ASSESSORS directory if it exists, otherwise None.
    """
    
    assessors_dir = os.path.join(patient_directory, 'ASSESSORS')
    if not os.path.exists(assessors_dir):
        logging.info("No segmentation exists")
        return None
    logging.info(f"'ASSESSORS' folder exists: {assessors_dir}")
    return assessors_dir


def get_segmentations_from_assessors_path(assessors_path: str) -> Dict[str, Dict[str, Any]]:
    """
    This function searches the ASSESSORS folder and creates a dictionary of all segmentations.
    
    Parameters:
    - assessors_path (str): The path to the ASSESSORS directory.
    
    Returns:
    - Dict[str, Dict[str, Any]]: A dictionary with the name of the segmentor and datetime of segmentation as keys. 
      Each key maps to another dictionary containing 'created_by', 'created_time', 'dicom_name', and 'dicom_fullpath'.
    """
    segmentation_paths = [d for d in os.listdir(assessors_path) if os.path.isdir(os.path.join(assessors_path, d))]
    segmentations = {}

    for seg in segmentation_paths:
        seg_dir = os.path.join(assessors_path, seg, 'SEG')
        if os.path.exists(seg_dir):
            files = os.listdir(seg_dir)
            
            # Find XML files
            xml_files = [f for f in files if f.endswith('.xml')]
            
            for xml_file in xml_files:
                xml_path = os.path.join(seg_dir, xml_file)
                try:
                    # Parse the XML file
                    tree = ET.parse(xml_path)
                    root = tree.getroot()
                    
                    # Extract createdBy and createdTime
                    entry = root.find('.//cat:entry', namespaces={'cat': 'http://nrg.wustl.edu/catalog'})
                    if entry is not None:
                        created_by = entry.get('createdBy')
                        created_time = entry.get('createdTime')
                        dicom_name = entry.get('name')
                        dicom_fullpath = os.path.join(seg_dir, dicom_name)
                        
                        # Save to dictionary
                        key = f"{created_by}>>{created_time}"
                        segmentations[key] = {
                            'created_by': created_by,
                            'created_time': created_time,
                            'dicom_name': dicom_name,
                            'dicom_fullpath': dicom_fullpath
                        }
                        logging.info(segmentations[key])
                except ET.ParseError as e:
                    logging.error(f"Error parsing XML file {xml_file}: {e}")
                except Exception as e:
                    logging.error(f"Unexpected error: {e}")

    return segmentations

ass_path = assessors_path(patient_directory)                    
all_segmentation_dic = get_segmentations_from_assessors_path(ass_path)


## Select the desired segmentation

In [ ]:
def select_segmentation_from_valid(all_segmentation_dic: Dict[str, Dict[str, Any]], default_selected_segmentation: str = None) -> Optional[str]:
    """
    This function allows the user to select a segmentation from the dictionary of segmentations.
    
    Parameters:
    - all_segmentation_dic (Dict[str, Dict[str, Any]]): Dictionary containing segmentations.
    - default_selected_segmentation (str, optional): Default selected segmentation. Defaults to None.
    
    Returns:
    - Optional[str]: The full path to the selected DICOM file, or None if not found.
    """
    valid_options_to_select = list(all_segmentation_dic.keys())
    len_options_to_select = len(valid_options_to_select)
    
    if len_options_to_select == 0:
        logging.warning("No valid segmentations available.")
        return None
    
    if not default_selected_segmentation:
        show_options_to_select = "\n".join([f"{i}: {valid_options_to_select[i]}" for i in range(len_options_to_select)])
        print(f"Available segmentations:\n{show_options_to_select}")
        selected_segmentation = input("Enter the index or name of your desired segmentation to visualize: ")
    else:
        selected_segmentation = default_selected_segmentation
    
    try:
        selected_segmentation_index = int(selected_segmentation)
        if selected_segmentation_index < 0 or selected_segmentation_index >= len_options_to_select:
            raise IndexError
        selected_segmentation_name = valid_options_to_select[selected_segmentation_index]
    except (ValueError, IndexError):
        selected_segmentation_name = selected_segmentation
        
    segmentation_info = all_segmentation_dic.get(selected_segmentation_name)
    if segmentation_info:
        logging.info(f"Selected SEG: {selected_segmentation_name}")
        logging.info(f"Path to selected SEG: {segmentation_info.get('dicom_fullpath')}")
        return segmentation_info.get('dicom_fullpath')
    else:
        logging.error(f"Selected segmentation '{selected_segmentation_name}' not found in the dictionary.")
        return None

# Example usage
selected_SEGdicom_fullpath = select_segmentation_from_valid(all_segmentation_dic,default_selected_segmentation=2)
selected_SEGdicom_data = read_dicom(selected_SEGdicom_fullpath)

### Extract data of each segmentation

In [6]:
def read_selected_segmentation_for_widgets(selected_segmentation_name, all_segmentation_dic):
    segmentation_info = all_segmentation_dic.get(selected_segmentation_name)
    if segmentation_info:
        logging.info(f"Selected SEG: {selected_segmentation_name}")
        logging.info(f"Path to selected SEG: {segmentation_info.get('dicom_fullpath')}")
        return segmentation_info.get('dicom_fullpath')
    else:
        logging.error(f"Selected segmentation '{selected_segmentation_name}' not found in the dictionary.")
        return None

### Extract segmentation file data

# Series description

In [ ]:
def get_segmentation_description(segmentation_dicom) -> str:
    """
    Extract a list of Referenced SOP Instance UIDs from a DICOM segmentation dataset.

    Parameters:
        segmentation_dicom (pydicom.dataset.Dataset): The DICOM dataset containing segmentation data.

    Returns:
        list: A list of Referenced SOP Instance UIDs.
    """
    try:
        segmentation_description = get_nested_element(segmentation_dicom, [(0x0008, 0x103E)]).value
        logging.info(f"segmentation_description: {segmentation_description}")
        return segmentation_description
    except (KeyError, ValueError) as e:
        logging.error(f"Error extracting segmentation_description: {e}")

segmentation_description = get_segmentation_description(selected_SEGdicom_data)
segmentation_description

#### Reference Series UID

In [ ]:
def get_referenced_series_UID(segmentation_dicom) -> str:
    """
    Extract a list of Referenced SOP Instance UIDs from a DICOM segmentation dataset.

    Parameters:
        segmentation_dicom (pydicom.dataset.Dataset): The DICOM dataset containing segmentation data.

    Returns:
        list: A list of Referenced SOP Instance UIDs.
    """
    try:
        referenced_series_sequence = get_nested_element(segmentation_dicom, [(0x0008, 0x1115)])
        for series_instance in referenced_series_sequence:
            if 'SeriesInstanceUID' in series_instance:
                Ref_series_UID = series_instance.SeriesInstanceUID
                logging.info(f"Referenced Series Instance UID: {Ref_series_UID}")
                return Ref_series_UID
    except (KeyError, ValueError) as e:
        logging.error(f"Error extracting Referenced Series Instance UID: {e}")

Ref_series_UID = get_referenced_series_UID(selected_SEGdicom_data)

#### Segmentation number-name dict map

In [ ]:

def create_segment_number_to_label_map(segmentation_dicom) -> dict:
    """
    Create a dictionary mapping Segment Number to Segment Label from a DICOM segmentation object.

    Parameters:
        segmentation_dicom (pydicom.dataset.Dataset): The DICOM dataset containing segmentation data.

    Returns:
        dict: A dictionary mapping Segment Number (int) to Segment Label (str).
    """
    segment_map = {}
    try:
        segment_sequence = get_nested_element(segmentation_dicom, [(0x0062, 0x0002)])
        for item in segment_sequence:
            if (0x0062, 0x0004) in item and (0x0062, 0x0005) in item:
                segment_number = item[(0x0062, 0x0004)].value
                segment_label = item[(0x0062, 0x0005)].value
                segment_map[segment_number] = segment_label
        logging.info(f"Segment map: {segment_map}")
    except (KeyError, ValueError) as e:
        logging.error(f"Error creating segment map: {e}")
    return segment_map

segment_map = create_segment_number_to_label_map(selected_SEGdicom_data)

#### slice data: RefSOPUID + Segment number (and label) + pixel data

In [ ]:
def get_segmentation_data_including_RefSOPUID_refSegNum_pixelData(segmentation_dicom, segment_map):
    """
    Reads a segmentation DICOM file and creates a dictionary for each slice.

    Parameters:
    - segmentation_dicom: The DICOM dataset for the segmentation.
    - segment_map: Dictionary to map segment numbers to segment labels.

    Returns:
    - Dictionary with segment labels as keys and lists of dictionaries as values. Each dictionary contains 'pixel_data' and 'sop_instance_uid'.
    """
    segmentation_data = {}

    try:
        # Get the number of frames
        num_frames = segmentation_dicom.NumberOfFrames
        logging.info(f"Number of frames in the DICOM: {num_frames}")

        # Get pixel data
        pixel_data = segmentation_dicom.pixel_array

        # Get referenced instance UID and segmentation number for each frame
        for frame_index in range(num_frames):
            try:
                frame = segmentation_dicom.PerFrameFunctionalGroupsSequence[frame_index]
                referenced_sop_instance_uid = frame.DerivationImageSequence[0].SourceImageSequence[0].ReferencedSOPInstanceUID
                image_position_patient=frame.PlanePositionSequence[0].ImagePositionPatient
                segment_number = frame.SegmentIdentificationSequence[0].ReferencedSegmentNumber
                
                if segment_number in segment_map:
                    segment_label = segment_map[segment_number]
                    slice_info = {
                        'pixel_data': pixel_data[frame_index],
                        'sop_instance_uid': referenced_sop_instance_uid,
                        'image_position_patient': image_position_patient
                    }
                    if segment_label not in segmentation_data:
                        segmentation_data[segment_label] = []
                    segmentation_data[segment_label].append(slice_info)
                else:
                    logging.warning(f"Segment number {segment_number} not found in segment_map.")
                    
            except Exception as e:
                logging.error(f"Error processing frame {frame_index}: {e}")

        # Log the count of slices for each segment label
        for segment_label, slices in segmentation_data.items():
            logging.info(f"Segment '{segment_label}': {len(slices)} slices")

    except AttributeError as e:
        logging.error(f"Error reading DICOM attributes: {e}")
    except Exception as e:
        logging.error(f"Unexpected error: {e}")

    return segmentation_data

slice_info_list = get_segmentation_data_including_RefSOPUID_refSegNum_pixelData(selected_SEGdicom_data,segment_map)
slice_info_list

## All segmentations in directory

In [11]:
import pandas as pd
from tqdm.notebook import tqdm
import os
import pickle
import re

def sanitize_filename(filename):
    # Remove invalid characters and replace with underscore
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

def find_all_segmentations_v2(project_directory, save_excel_path=None, save_pkl_path=None):
    patient_directories = list_directory_folders(project_directory)
    all_segmentation_df = pd.DataFrame(columns=['patient_directory', 'segmentation_name', 'segmentation_filepath'])
    
    for patient_directory in tqdm(patient_directories):
        try:
            ass_path = assessors_path(patient_directory)                    
            all_segmentation_dic = get_segmentations_from_assessors_path(ass_path)
            for selected_segmentation in all_segmentation_dic.keys():
                
                selected_SEGdicom_fullpath = select_segmentation_from_valid(all_segmentation_dic, default_selected_segmentation=selected_segmentation)
                selected_SEGdicom_data = read_dicom(selected_SEGdicom_fullpath)

                Ref_series_UID = get_referenced_series_UID(selected_SEGdicom_data)
                segment_map = create_segment_number_to_label_map(selected_SEGdicom_data)
                slice_info_list = get_segmentation_data_including_RefSOPUID_refSegNum_pixelData(selected_SEGdicom_data, segment_map)
                segmentation_description = get_segmentation_description(selected_SEGdicom_data)
                                
                new_row = {
                    'patient_directory': patient_directory,
                    'segmentation_name': selected_segmentation,
                    'segmentation_filepath': selected_SEGdicom_fullpath,
                    'Ref_series_UID': Ref_series_UID,
                    'segmentation_description':segmentation_description
                }
                
                for segment_label, slices in slice_info_list.items():
                    new_row[segment_label] = len(slices)
                    
                    if save_pkl_path:
                        try:
                            # Sanitize directory and file names
                            patient_name = sanitize_filename(os.path.basename(patient_directory))
                            safe_segmentation_name = sanitize_filename(selected_segmentation)
                            safe_segment_label = sanitize_filename(segment_label)
                            
                            # Create directory structure for pickle files
                            pkl_dir = os.path.join(save_pkl_path, patient_name, safe_segmentation_name)
                            os.makedirs(pkl_dir, exist_ok=True)
                            
                            # Prepare the data to save in the pickle file
                            pkl_data = {
                                'Ref_series_UID': Ref_series_UID,
                                'segmentation_description': segmentation_description,
                                'case_number': patient_name,
                                'safe_segmentation_name': safe_segmentation_name,
                                'validation': "finalized",
                                'slices': slices,
                                
                                
                            }
                            
                            # Save pickle file
                            pkl_filename = f"{safe_segment_label}.pkl"
                            pkl_path = os.path.join(pkl_dir, pkl_filename)
                            with open(pkl_path, 'wb') as f:
                                pickle.dump(pkl_data, f)
                        except Exception as pkl_error:
                            print(f"Error saving pickle for {patient_directory}, {selected_segmentation}, {segment_label}: {str(pkl_error)}")
                
        except Exception as e:
            new_row = {
                'patient_directory': patient_directory,
                'segmentation_name': str(e),
                'segmentation_filepath': str(e),
                'Ref_series_UID': str(e),
                'segmentation_description':str(e)
            }
            all_segmentation_df = pd.concat([all_segmentation_df, pd.DataFrame([new_row])], ignore_index=True)
    
    if save_excel_path:
        all_segmentation_df.to_excel(save_excel_path, index=False)
    
    return all_segmentation_df

In [12]:
import pandas as pd
from tqdm.notebook import tqdm
import os
import pickle
import re

def sanitize_filename(filename):
    # Remove invalid characters and replace with underscore
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

def find_all_segmentations_v3(project_directory, save_excel_path=None, save_pkl_path=None):
    patient_directories = list_directory_folders(project_directory)
    all_segmentation_df = pd.DataFrame(columns=['patient_directory', 'segmentation_name', 'segmentation_filepath'])
    
    for patient_directory in tqdm(patient_directories):
        try:
            ass_path = assessors_path(patient_directory)                    
            all_segmentation_dic = get_segmentations_from_assessors_path(ass_path)
            for selected_segmentation in all_segmentation_dic.keys():
                
                selected_SEGdicom_fullpath = select_segmentation_from_valid(all_segmentation_dic, default_selected_segmentation=selected_segmentation)
                selected_SEGdicom_data = read_dicom(selected_SEGdicom_fullpath)

                Ref_series_UID = get_referenced_series_UID(selected_SEGdicom_data)
                segment_map = create_segment_number_to_label_map(selected_SEGdicom_data)
                slice_info_list = get_segmentation_data_including_RefSOPUID_refSegNum_pixelData(selected_SEGdicom_data, segment_map)
                segmentation_description = get_segmentation_description(selected_SEGdicom_data)
                                
                new_row = {
                    'patient_directory': patient_directory,
                    'segmentation_name': selected_segmentation,
                    'segmentation_filepath': selected_SEGdicom_fullpath,
                    'Ref_series_UID': Ref_series_UID,
                    'segmentation_description':segmentation_description
                }
                
                for segment_label, slices in slice_info_list.items():
                    new_row[segment_label] = len(slices)
                    
                    if save_pkl_path:
                        try:
                            # Sanitize directory and file names
                            patient_name = sanitize_filename(os.path.basename(patient_directory))
                            safe_segmentation_name = sanitize_filename(selected_segmentation)
                            safe_segment_label = sanitize_filename(segment_label)
                            safe_description = sanitize_filename(segmentation_description)
                            
                            # Create directory structure for pickle files
                            pkl_dir = os.path.join(save_pkl_path, patient_name, f"{safe_description}__{safe_segmentation_name}")
                            os.makedirs(pkl_dir, exist_ok=True)
                            
                            # Prepare the data to save in the pickle file
                            pkl_data = {
                                'Ref_series_UID': Ref_series_UID,
                                'segmentation_description': segmentation_description,
                                'case_number': patient_name,
                                'safe_segmentation_name': safe_segmentation_name,
                                'validation': "finalized",
                                'slices': slices,
                                
                                
                            }
                            
                            # Save pickle file
                            pkl_filename = f"{safe_segment_label}.pkl"
                            pkl_path = os.path.join(pkl_dir, pkl_filename)
                            with open(pkl_path, 'wb') as f:
                                pickle.dump(pkl_data, f)
                        except Exception as pkl_error:
                            print(f"Error saving pickle for {patient_directory}, {selected_segmentation}, {segment_label}: {str(pkl_error)}")
                
        except Exception as e:
            new_row = {
                'patient_directory': patient_directory,
                'segmentation_name': str(e),
                'segmentation_filepath': str(e),
                'Ref_series_UID': str(e),
                'segmentation_description':str(e)
            }
            all_segmentation_df = pd.concat([all_segmentation_df, pd.DataFrame([new_row])], ignore_index=True)
    
    if save_excel_path:
        all_segmentation_df.to_excel(save_excel_path, index=False)
    
    return all_segmentation_df

# Run: Save pickels

In [ ]:
# save_excel_path=r"D:\BATCH1_seg_cleaned_E2\all_segmentations.xlsx"
# save_pkl_path=r"D:\BATCH1_seg_cleaned_E2"
# # all_segmentation_df=find_all_segmentations_v2(project_directory, save_excel_path=save_excel_path, save_pkl_path=save_pkl_path)
# all_segmentation_df=find_all_segmentations_v3(project_directory, save_excel_path=save_excel_path, save_pkl_path=save_pkl_path)
# all_segmentation_df

### show pickle content

In [ ]:
### show pickle content

import pickle

# Replace 'your_file.pkl' with the path to your pickle file
pickle_file_path = r"C:\Users\LEGION\Desktop\BATCH1_seg_cleaned_E2_validated\2105\Salmanipour__AlirezaSoleimanpour__2024-01-24T09_20_47.856\P.pkl"

# Open the pickle file and load the dictionary
with open(pickle_file_path, 'rb') as file:
    data = pickle.load(file)

# Display the content of the dictionary

data



In [ ]:
import pickle

# Replace 'your_file.pkl' with the path to your pickle file
pickle_file_path = r"D:\BATCH1_seg_cleaned_E2\1000034\FFF_elham_PAS1__elham_gpteam__2024-01-24T15_16_47.347\P.pkl"

# Open the pickle file and load the dictionary
with open(pickle_file_path, 'rb') as file:
    data = pickle.load(file)

# Display the content of the dictionary

print(data.items())



# Pickle to final pickle

In [43]:
import os
import xml.etree.ElementTree as ET
import logging

def extract_series_data(patient_directory):
    """
    Extracts series data from XML files within the SCANS directory of the given patient directory.

    Parameters:
    - patient_directory: Path to the patient's directory containing the SCANS folder.

    Returns:
    - Dictionary containing series data.
    """
    scans_dir = os.path.join(patient_directory, 'SCANS')
    series_folders = [d for d in os.listdir(scans_dir) if os.path.isdir(os.path.join(scans_dir, d))]
    
    series_data = {}

    for series in series_folders:
        series_path = os.path.join(scans_dir, series, 'DICOM')
        xml_files = [f for f in os.listdir(series_path) if f.endswith('.xml')]

        if not xml_files:
            logging.warning(f"No XML files found in series directory: {series_path}")
            continue

        # Assuming there's only one XML file per series directory
        xml_file = xml_files[0]
        xml_path = os.path.join(series_path, xml_file)

        try:
            tree = ET.parse(xml_path)
            root = tree.getroot()

            # Extract original_RefSeriesUID
            ref_series_uid = root.attrib.get('UID', None)
            if not ref_series_uid:
                logging.warning(f"No UID found in XML root for series {series}")
                continue

            # Extract original_RefInstanceUID_dict
            ref_instance_uid_dict = {}
            for entry in root.findall('.//cat:entry', namespaces={'cat': 'http://nrg.wustl.edu/catalog'}):
                uid = entry.attrib.get('UID')
                uri = entry.attrib.get('URI')
                if uid and uri:
                    ref_instance_uid_dict[uid] = os.path.join(series_path, uri)

            series_data[series] = {
                'original_RefSeriesUID': ref_series_uid,
                'original_RefInstanceUID_dict': ref_instance_uid_dict
            }
        except ET.ParseError as e:
            logging.error(f"Error parsing XML file {xml_file}: {e}")
        except Exception as e:
            logging.error(f"Unexpected error processing series {series}: {e}")
            
    logging.info(f"Successfully read the XML file for {patient_directory}. Series and slice counts:")
    for key, value in series_data.items():
        original_RefInstanceUID_dict = value.get('original_RefInstanceUID_dict',{})
        logging.info(f"    {key}: {len(original_RefInstanceUID_dict)}")    
        
    return series_data

def get_path_to_matched_original_series(original_series_data,Ref_series_UID, patient_directory):
    for original_folder, original_info_dic in original_series_data.items():
        if original_info_dic['original_RefSeriesUID']==Ref_series_UID:
            logging.info(f"Successfully find a match for series UIDs in folder: '{original_folder}'")
            
            matched_series_folder = os.path.join(patient_directory,"SCANS", original_folder,"DICOM")
 
    if matched_series_folder:
        logging.info(" The matched series of original image created successfully.")
        return  matched_series_folder
    else:
        logging.warning("No matched series UID was ")
        
        
def read_origignal_dicom_pixel_data(directory_path):
    """
    Reads pixel data and referenced SOP Instance UID from all DICOM files in the given directory.

    Parameters:
    - directory_path: Path to the directory containing DICOM files.

    Returns:
    - List of dictionaries with 'pixel_data', 'image_position_patient', and 'instance_number'.
    """
    dicom_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.dcm')]
    dicom_data_list = []
    
    for dicom_file in dicom_files:
        try:
            dicom_data = pydicom.dcmread(dicom_file)
            sop_instance_uid = dicom_data.SOPInstanceUID
            image_position_patient = dicom_data.ImagePositionPatient
            pixel_data = dicom_data.pixel_array
            instance_number = dicom_data.InstanceNumber if hasattr(dicom_data, 'InstanceNumber') else 0
            
            dicom_data_list.append({
                'pixel_data': pixel_data,
                'sop_instance_uid': sop_instance_uid,
                'image_position_patient': image_position_patient,
                'instance_number': instance_number
            })
            

            Pixel_Spacing = dicom_data.PixelSpacing

            Slice_Thickness= dicom_data.SliceThickness

            Series_Description = dicom_data.SeriesDescription
            
            dicom_meta_data = {
                "Pixel_Spacing": Pixel_Spacing,
                "Slice_Thickness": Slice_Thickness,
                "Series_Description": Series_Description
            }
        except Exception as e:
            logging.error(f"Error reading DICOM file '{dicom_file}': {e}")
            dicom_meta_data=None

    
    # Sort by InstanceNumber
    dicom_data_list.sort(key=lambda x: x['instance_number'])
    
    return dicom_data_list, dicom_meta_data

def merge_dictionaries_bySOP(original_dic, segment_dic):
    """
    Merges two dictionaries based on the SOP Instance UID.

    Parameters:
    - original_dic: List of dictionaries with 'pixel_data', 'sop_instance_uid', and other keys.
    - segment_dic: List of dictionaries with 'pixel_data', 'sop_instance_uid', and other keys.

    Returns:
    - Merged list of dictionaries.
    """
    merged_data = []
    segment_lookup = {item['sop_instance_uid']: item['pixel_data'] for item in segment_dic}
    
    for original_item in original_dic:
        sop_instance_uid = original_item['sop_instance_uid']
        segmentation_pixel = segment_lookup.get(sop_instance_uid, None)
        
        merged_data.append({
            'sop_instance_uid': sop_instance_uid,
            'image_position_patient': original_item['image_position_patient'],
            'original_pixel': original_item['pixel_data'],
            'segmentation_pixel': segmentation_pixel,
            'instance_number': original_item.get('instance_number')
        })
    
    return merged_data

def merge_dictionaries_byPatientPosition(original_dic, segment_dic):
    """
    Merges two dictionaries based on the SOP Instance UID.

    Parameters:
    - original_dic: List of dictionaries with 'pixel_data', 'sop_instance_uid', and other keys.
    - segment_dic: List of dictionaries with 'pixel_data', 'sop_instance_uid', and other keys.

    Returns:
    - Merged list of dictionaries.
    """
    merged_data = []
    segment_lookup = {str(item['image_position_patient']): item['pixel_data'] for item in segment_dic}
    
    for original_item in original_dic:
        image_position_patient = str(original_item['image_position_patient'])
        segmentation_pixel = segment_lookup.get(image_position_patient, None)
        
        merged_data.append({
            'sop_instance_uid': original_item['sop_instance_uid'], 
            'image_position_patient': image_position_patient,
            'original_pixel': original_item['pixel_data'],
            'segmentation_pixel': segmentation_pixel,
            'instance_number': original_item.get('instance_number')
        })
    
    return merged_data



In [ ]:
import os
import pickle
import logging

# Function to process the segmentation and merge with original CT
def process_segmentation_pickle(pickle_path, project_directory, save_pkl_directory):
    """
    Processes the segmentation pickle file and merges the segmentation with the original CT series.

    Parameters:
    - pickle_path: Path to the segmentation pickle file.
    - project_directory: Directory containing patient folders.
    - save_directory: Directory to save the updated pickle with merged data.

    Returns:
    - Path to the updated pickle file.
    """
    
    # Load the segmentation pickle file
    with open(pickle_path, 'rb') as f:
        pkl_data = pickle.load(f)

    # Extract case_number and Ref_series_UID from the pickle data
    case_number = pkl_data.get('case_number')
    Ref_series_UID = pkl_data.get('Ref_series_UID')

    # Find the patient folder using case_number
    patient_directory = os.path.join(project_directory, case_number)
    if not os.path.exists(patient_directory):
        logging.error(f"Patient directory {patient_directory} not found.")
        return None

    # Use extract_series_data to get series information
    original_series_data = extract_series_data(patient_directory)

    # Find the matched series for Ref_series_UID
    matched_series_folder = get_path_to_matched_original_series(original_series_data, Ref_series_UID, patient_directory)
    if not matched_series_folder:
        logging.error(f"No matching series found for UID: {Ref_series_UID}")
        return None

    
    dicom_data_list, dicom_meta_data = read_origignal_dicom_pixel_data(matched_series_folder)
    
    # Merge the segmentation slices with the original CT data using merge_ct_and_segmentation function
    #merged_data = merge_dictionaries_bySOP(dicom_data_list, pkl_data['slices'])
    merged_data = merge_dictionaries_byPatientPosition(dicom_data_list, pkl_data['slices'])
    
    print(f"number of slices in merged_data: {len(merged_data)}")
    merged_slices_count=0
    for dic in merged_data:
        if dic['segmentation_pixel'] is not None:
            merged_slices_count+=1
    print(f"number of segmentation (merged) slices merged_data {merged_slices_count}")
    merged_slices_count

    
    dicom_meta_data['case_number']=case_number
    dicom_meta_data['Ref_series_UID']=Ref_series_UID
    dicom_meta_data['validation_level']='3_finalized'
    
    processed_output_dic = {
        "dicom_meta_data": dicom_meta_data,
        "merged_slices": merged_data
    }

    if save_pkl_directory:
        # Save the merged data in a new pickle file
        case_folder = os.path.basename(os.path.dirname(os.path.dirname(pickle_path)))
        subfolder = os.path.basename(os.path.dirname(pickle_path))
        filename = os.path.basename(pickle_path)
        
        updated_pickle_path = os.path.join(save_pkl_directory, case_folder, subfolder, filename)
        
        # Ensure the directory exists
        os.makedirs(os.path.dirname(updated_pickle_path), exist_ok=True)

        with open(updated_pickle_path, 'wb') as f:
            pickle.dump(processed_output_dic, f)
        logging.info(f"Updated pickle file saved at: {updated_pickle_path}")

    return processed_output_dic

# Example usage
pickle_file_path = r"D:\BATCH1_seg_cleaned_E2\1000034\FFF_elham_PAS1__elham_gpteam__2024-01-24T15_16_47.347\P.pkl"
project_dir = r"D:\PanCanAID-Batch1-202408"
save_pkl_directory = r'D:\BATCH1_seg_merged'

# desired save location r"D:\BATCH1_seg_merged\1000034\FFF_elham_PAS1__elham_gpteam__2024-01-24T15_16_47.347\P.pkl"

processed_output_dic = process_segmentation_pickle(pickle_file_path, project_dir, save_pkl_directory)
processed_output_dic


In [ ]:
merged_data

In [ ]:
import os
import glob

def process_multiple_pickles(pickle_names, source_directory, project_directory, save_pkl_directory):
    """
    Process multiple pickle files based on provided names.

    Parameters:
    - pickle_names: List of strings, each representing a pickle file name to process.
    - source_directory: Directory to search for pickle files.
    - project_directory: Directory containing patient folders (passed to process_segmentation_pickle).
    - save_pkl_directory: Directory to save processed pickles (passed to process_segmentation_pickle).

    Returns:
    - A list of processed output dictionaries.
    """
    processed_outputs = []

    # Recursively search for all .pkl files in the source_directory
    all_pickles = glob.glob(os.path.join(source_directory, '**', '*.pkl'), recursive=True)

    for pickle_name in tqdm(pickle_names):
        # Find matching pickle files
        matching_pickles = [p for p in all_pickles if os.path.basename(p) == pickle_name]

        if not matching_pickles:
            logging.warning(f"No pickle file found with name: {pickle_name}")
            continue

        for pickle_path in tqdm(matching_pickles):
            logging.info(f"Processing pickle file: {pickle_path}")
            
            try:
                processed_output = process_segmentation_pickle(pickle_path, project_directory, save_pkl_directory)
                if processed_output:
                    processed_outputs.append(processed_output)
                    logging.info(f"Successfully processed: {pickle_path}")
                else:
                    logging.warning(f"Failed to process: {pickle_path}")
            except Exception as e:
                logging.error(f"Error processing {pickle_path}: {str(e)}")

    return processed_outputs

# Example usage:
pickle_names = ["P.pkl", "p.pkl", 'Pancreas.pkl', 'pancreas.pkl', 'm.pkl','M.pkl', 'mass.pkl', 'Mass.pkl', 'mpd.pkl' ,'MPD.pkl' ]
source_directory = r"D:\Batch1_20240830\BATCH1_seg_clean_semiValid"
project_directory = r"D:\Batch1_20240830\PanCanAID-Batch1-202408"
save_pkl_directory = r"D:\Batch1_20240830\BATCH1_seg_merged_Ps"

results = process_multiple_pickles(pickle_names, source_directory, project_directory, save_pkl_directory)
print(f"Processed {len(results)} pickle files successfully.")

# Play with a pickle

### show pickle content


In [ ]:
import pickle

# Replace 'your_file.pkl' with the path to your pickle file
pickle_file_path = r"C:\Users\LEGION\Desktop\BATCH1_seg_clean_semiValid\2128\AAA_validated__admin__2024-09-10T07_56_44.721\G.pkl"
# Open the pickle file and load the dictionary
with open(pickle_file_path, 'rb') as file:
    data = pickle.load(file)

# Display the content of the dictionary

data


## modify pickle validation

In [62]:
import pickle
import tkinter as tk
from tkinter import simpledialog, messagebox

def modify_validation_in_pickle(pickle_path):
    """
    Load a pickle file, display a popup to modify the 'validation' item,
    and save the modified pickle file.

    Parameters:
    - pickle_path: Path to the pickle file to modify.

    Returns:
    - The modified dictionary if successful, None otherwise.
    """
    try:
        # Load the pickle file
        with open(pickle_path, 'rb') as f:
            data = pickle.load(f)

        # Check if 'validation' exists in the dictionary
        if 'validation' not in data:
            messagebox.showerror("Error", "'validation' key not found in the pickle file.")
            return None

        # Create the main window
        root = tk.Tk()
        root.withdraw()  # Hide the main window

        # Show a dialog to get the new validation value
        current_validation = data['validation']
        new_validation = simpledialog.askstring("Modify Validation", 
                                                f"Current validation: {current_validation}\n"
                                                "Enter new validation value:",
                                                initialvalue=current_validation)

        # If user cancels, return None
        if new_validation is None:
            return None

        # Update the validation in the dictionary
        data['validation'] = new_validation

        # Save the modified dictionary back to the pickle file
        with open(pickle_path, 'wb') as f:
            pickle.dump(data, f)

        messagebox.showinfo("Success", f"Validation updated to: {new_validation}")
        return data

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")
        return None

# Example usage
# pickle_path = r"path\to\your\pickle\file.pkl"
# modified_data = modify_validation_in_pickle(pickle_path)

# if modified_data:
#     print("Pickle file successfully modified.")
#     print(f"New validation value: {modified_data['validation']}")
# else:
#     print("Failed to modify pickle file.")


In [63]:
pickle_paths=[
    r"C:\Users\LEGION\Desktop\BATCH1_seg_cleaned_E2_validated\2166\AHMAD SHOJA__AhmadShoja__2024-01-05T18_01_49.533\M.pkl",
    r"C:\Users\LEGION\Desktop\BATCH1_seg_cleaned_E2_validated\2166\AHMAD SHOJA__AhmadShoja__2024-01-05T18_01_49.533\MPD.pkl",
    r"C:\Users\LEGION\Desktop\BATCH1_seg_cleaned_E2_validated\2166\AHMAD SHOJA__AhmadShoja__2024-01-05T18_01_49.533\P.pkl"
]

In [ ]:
# for pickle_path in pickle_paths:
#     modified_data = modify_validation_in_pickle(pickle_path)

#     if modified_data:
#         print("Pickle file successfully modified.")
#         print(f"New validation value: {modified_data['validation']}")
#     else:
#         print("Failed to modify pickle file.")

## 2D

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, fixed, widgets
from IPython.display import display

def visualize_segmentation_pickle(pickle_file_path):
    # Load the pickle file
    with open(pickle_file_path, 'rb') as f:
        all_segmentation_data = pickle.load(f)
        
    segmentation_data = all_segmentation_data['slices']
    # Check if the data is a list of dictionaries
    if not isinstance(segmentation_data, list) or not all(isinstance(item, dict) for item in segmentation_data):
        raise ValueError("The pickle file should contain a list of dictionaries.")
    
    # Extract pixel data and positions
    pixel_data_list = [item['pixel_data'] for item in segmentation_data]
    positions = [item['image_position_patient'] for item in segmentation_data]
    
    # Create a 3D volume
    volume = np.stack(pixel_data_list, axis=0)
    
    # Function to view a slice
    def view_slice(axis, slice_num):
        plt.figure(figsize=(10, 10))
        if axis == 'Axial':
            plt.imshow(volume[slice_num, :, :], cmap='gray')
            plt.title(f'Axial Slice {slice_num}')
        elif axis == 'Coronal':
            plt.imshow(volume[:, slice_num, :], cmap='gray')
            plt.title(f'Coronal Slice {slice_num}')
        else:  # Sagittal
            plt.imshow(volume[:, :, slice_num], cmap='gray')
            plt.title(f'Sagittal Slice {slice_num}')
        plt.colorbar(label='Pixel Value')
        plt.axis('off')
        plt.show()
    
    # Create interactive widget
    axis_widget = widgets.Dropdown(options=['Axial', 'Coronal', 'Sagittal'], value='Axial', description='View:')
    slice_widget = widgets.IntSlider(min=0, max=volume.shape[0]-1, step=1, value=volume.shape[0]//2, description='Slice:')
    
    # Update slice widget based on selected axis
    def update_slice_widget(*args):
        axis = axis_widget.value
        if axis == 'Axial':
            slice_widget.max = volume.shape[0] - 1
        elif axis == 'Coronal':
            slice_widget.max = volume.shape[1] - 1
        else:  # Sagittal
            slice_widget.max = volume.shape[2] - 1
        slice_widget.value = slice_widget.max // 2
    
    axis_widget.observe(update_slice_widget, 'value')
    
    # Display interactive widget
    interactive_plot = interactive(view_slice, axis=axis_widget, slice_num=slice_widget)
    display(interactive_plot)
    
    # Print some information about the segmentation
    print(f"Number of slices: {len(segmentation_data)}")
    print(f"Dimensions of volume: {volume.shape}")
    print(f"Range of pixel values: {np.min(volume)} to {np.max(volume)}")

# Example usage:
pickle_path=r"C:\Users\LEGION\Desktop\BATCH1_seg_cleaned_E2_validated\2282\ahmad shoja__AhmadShoja__2024-01-05T16_02_14.157\p.pkl"
visualize_segmentation_pickle(pickle_path)

## 3D

In [ ]:
import pickle
import numpy as np
import vtk
from vtk.util import numpy_support

def create_3d_visualization(pickle_file_path, smoothing=True, morphology=True, advanced_smoothing=True):
    # Load the pickle file
    with open(pickle_file_path, 'rb') as f:
        segmentation_data = pickle.load(f)
    
    # Extract pixel data and positions
    pixel_data_list = [item['pixel_data'] for item in segmentation_data]
    positions = [item['image_position_patient'] for item in segmentation_data]
    
    # Calculate slice spacing
    slice_positions = [pos[2] for pos in positions]  # Assuming Z is the slice direction
    slice_spacing = np.mean(np.diff(sorted(slice_positions)))
    
    # Get dimensions of a single slice
    slice_shape = pixel_data_list[0].shape
    
    # Create a 3D volume with correct spacing
    volume = np.stack(pixel_data_list, axis=0)
    
    # Create VTK image data
    vtk_image = vtk.vtkImageData()
    vtk_image.SetDimensions(slice_shape[1], slice_shape[0], len(pixel_data_list))
    vtk_image.SetSpacing(1.0, 1.0, slice_spacing)  # Assuming 1mm pixel spacing in X and Y
    vtk_image.SetOrigin(positions[0][0], positions[0][1], positions[0][2])  # Set origin to first slice position
    
    # Flatten the numpy array and convert to VTK array
    vtk_array = numpy_support.numpy_to_vtk(volume.ravel(), deep=True, array_type=vtk.VTK_UNSIGNED_CHAR)
    vtk_image.GetPointData().SetScalars(vtk_array)
    
    if morphology:
        # Apply morphological operations
        morph_filter = vtk.vtkImageOpenClose3D()
        morph_filter.SetInputData(vtk_image)
        morph_filter.SetKernelSize(3, 3, 3)
        morph_filter.SetOpenValue(0)
        morph_filter.SetCloseValue(1)
        morph_filter.Update()
        vtk_image = morph_filter.GetOutput()
    
    # Create a surface from the image data
    contour = vtk.vtkMarchingCubes()
    contour.SetInputData(vtk_image)
    contour.SetValue(0, 1)  # Assuming binary segmentation, adjust if necessary
    contour.Update()
    
    if smoothing:
        # Apply initial smoothing
        smoother = vtk.vtkSmoothPolyDataFilter()
        smoother.SetInputConnection(contour.GetOutputPort())
        smoother.SetNumberOfIterations(15)
        smoother.SetRelaxationFactor(0.1)
        smoother.FeatureEdgeSmoothingOff()
        smoother.BoundarySmoothingOn()
        smoother.Update()
    else:
        smoother = contour
    
    if advanced_smoothing:
        # Apply windowed sinc smoothing
        sinc_smoother = vtk.vtkWindowedSincPolyDataFilter()
        sinc_smoother.SetInputConnection(smoother.GetOutputPort())
        sinc_smoother.SetNumberOfIterations(20)
        sinc_smoother.BoundarySmoothingOff()
        sinc_smoother.FeatureEdgeSmoothingOff()
        sinc_smoother.SetFeatureAngle(120.0)
        sinc_smoother.SetPassBand(0.001)
        sinc_smoother.NonManifoldSmoothingOn()
        sinc_smoother.NormalizeCoordinatesOn()
        sinc_smoother.Update()
        
        # Subdivide the surface
        subdivisionFilter = vtk.vtkLinearSubdivisionFilter()
        subdivisionFilter.SetInputConnection(sinc_smoother.GetOutputPort())
        subdivisionFilter.SetNumberOfSubdivisions(1)
        subdivisionFilter.Update()
        
        # Decimate (simplify) the mesh
        decimate = vtk.vtkDecimatePro()
        decimate.SetInputConnection(subdivisionFilter.GetOutputPort())
        decimate.SetTargetReduction(0.1)  # Reduce number of triangles by 10%
        decimate.PreserveTopologyOn()
        decimate.Update()
        
        # Final smoothing pass
        final_smoother = vtk.vtkSmoothPolyDataFilter()
        final_smoother.SetInputConnection(decimate.GetOutputPort())
        final_smoother.SetNumberOfIterations(10)
        final_smoother.SetRelaxationFactor(0.1)
        final_smoother.FeatureEdgeSmoothingOff()
        final_smoother.BoundarySmoothingOn()
        final_smoother.Update()
        
        # Create a mapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputConnection(final_smoother.GetOutputPort())
    else:
        # Create a mapper without advanced smoothing
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputConnection(smoother.GetOutputPort())
    
    # Create an actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    
    # Create a renderer and render window
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    
    # Create a render window interactor
    interactor = vtk.vtkRenderWindowInteractor()
    interactor.SetRenderWindow(render_window)
    
    # Add the actor to the scene
    renderer.AddActor(actor)
    renderer.SetBackground(0.1, 0.2, 0.4)  # Set background color
    
    # Reset camera and render
    renderer.ResetCamera()
    render_window.Render()
    
    # Start the interactor
    interactor.Start()

# Example usage:
pickle_path=r"D:\BATCH1_seg_cleaned\2248\elham_gpteam__2024-07-16T15_30_33.257\P.pkl"
create_3d_visualization(pickle_path, smoothing=True, morphology=True, advanced_smoothing=False)
